# 🎓 Clase 14 - CRUD seguro y manejo de errores en bases de datos

En este cuaderno vas a **fortalecer la gestión de datos en SQLite**, aprendiendo a hacer que tu sistema sea más seguro y robusto.

✅ Vas a mejorar tu implementación CRUD: Crear, Leer, Actualizar y Eliminar  

✅ Vas a prevenir errores y protegerte contra ataques de **inyección SQL**  

✅ Vas a usar **transacciones** y `try-except` para mantener la integridad de los datos

---

### ¿Qué vas a practicar hoy?

- Consultas SQL seguras usando parámetros (`?`)
- Validación de entradas para evitar datos erróneos
- Confirmaciones antes de eliminar registros
- Uso de `try`, `commit()` y `rollback()` para proteger los datos
- Aplicar todo esto en un sistema simple de gestión de alumnos

💡 *Nota*: No olvides que el archivo `.db` se guarda de forma temporal, y que tenés que descargarlo si querés conservarlo.

¡Empecemos!


# 🧑‍🎓 Crear la tabla alumnos

Vamos a trabajar con una nueva base de datos llamada `alumnos.db`, donde registraremos estudiantes con:

- `id`: número entero, clave primaria autoincremental
- `nombre`: texto obligatorio
- `edad`: número entero
- `email`: texto

Este será nuestro modelo de práctica para las operaciones CRUD seguras.


In [ ]:
import sqlite3

# Conectamos con la base
conexion = sqlite3.connect("alumnos.db")
cursor = conexion.cursor()

# Creamos la tabla si no existe
cursor.execute("""
CREATE TABLE IF NOT EXISTS alumnos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    edad INTEGER,
    email TEXT
)
""")

conexion.commit()
conexion.close()

print("Tabla 'alumnos' creada correctamente.")


# 🔍 ¿Qué hace este código?

📌 Crea el archivo `alumnos.db` si no existe.

📌 Usa `CREATE TABLE IF NOT EXISTS` para evitar errores si ya fue creada.

📌 Define un esquema de 4 campos, con `id` autogenerado.

🎯 Vamos a trabajar con esta tabla durante toda la clase.


# 🧑‍🎓 Agregar alumnos con parámetros seguros

Para evitar errores o ataques, usamos **parámetros (`?`)** en las consultas SQL.

En este ejemplo agregamos tres alumnos nuevos a la tabla.


In [ ]:
import sqlite3

# Lista de alumnos para insertar
nuevos = [
    ("Camila", 22, "camila@mail.com"),
    ("Julián", 19, "julian@mail.com"),
    ("Dana", 21, "dana@mail.com")
]

# Insertamos los registros
conexion = sqlite3.connect("alumnos.db")
cursor = conexion.cursor()

for nombre, edad, email in nuevos:
    cursor.execute("INSERT INTO alumnos (nombre, edad, email) VALUES (?, ?, ?)", (nombre, edad, email))

conexion.commit()
conexion.close()

print("Alumnos agregados correctamente.")


Alumnos agregados correctamente.


# 🔍 ¿Qué hace este código?

📌 Las consultas parametrizadas (`?`) protegen tu base y evitan errores.

📌 Los valores reales se pasan como tuplas (nombre, edad, email).

🎯 Probá modificar los nombres o agregar más alumnos a la lista.

# 🧑‍🎓 Consultar todos los alumnos

Podemos ver todos los registros de la tabla `alumnos` con `SELECT *`.

Recorremos el resultado y mostramos nombre, edad y correo.


In [ ]:
import sqlite3

# Consultamos los registros
conexion = sqlite3.connect("alumnos.db")
cursor = conexion.cursor()

cursor.execute("SELECT * FROM alumnos")
alumnos = cursor.fetchall()

conexion.close()

# Mostramos los resultados
print("Lista de alumnos registrados:")
for id_, nombre, edad, email in alumnos:
    print(f"{id_}. {nombre} ({edad} años) – {email}")


Lista de alumnos registrados:
1. Camila (22 años) – camila@mail.com
2. Julián (19 años) – julian@mail.com
3. Dana (21 años) – dana@mail.com


# 🔍 ¿Qué hace este código?

📌 `fetchall()` devuelve una lista de tuplas con los registros de la tabla.

📌 Cada tupla contiene los campos: `id`, `nombre`, `edad`, `email`.

🎯 Probá agregar más alumnos y volver a ejecutar este bloque.

# 🧑‍🎓 Modificar el email de un alumno

Para actualizar un campo específico usamos `UPDATE ... SET ... WHERE`.

Este ejemplo permite cambiar el email de un alumno, buscando por su `id`.


In [ ]:
import sqlite3

try:
    id_alumno = int(input("ID del alumno a modificar: "))
    nuevo_email = input("Nuevo correo electrónico: ").strip()

    conexion = sqlite3.connect("alumnos.db")
    cursor = conexion.cursor()

    cursor.execute("UPDATE alumnos SET email = ? WHERE id = ?", (nuevo_email, id_alumno))

    if cursor.rowcount > 0:
        print("Email actualizado correctamente.")
    else:
        print("No se encontró ningún alumno con ese ID.")

    conexion.commit()
    conexion.close()

except ValueError:
    print("El ID debe ser un número entero.")


ID del alumno a modificar: 1
Nuevo correo electrónico: pepe@gmail.com
Email actualizado correctamente.


# 🔍 ¿Qué hace este código?

📌 Usamos `input()` para obtener el ID y el nuevo email.

📌 Validamos que el ID sea un número entero (`try-except`).

📌 Verificamos si se actualizó algún registro usando `cursor.rowcount`.

🎯 Probá ingresar un ID inexistente o un email vacío y observá qué ocurre.


# 🧑‍🎓 Eliminar un alumno con confirmación

Para eliminar un alumno usamos `DELETE FROM`, pero antes pedimos confirmación al usuario.

Esto evita que borres registros por error.


In [ ]:
import sqlite3

try:
    id_alumno = int(input("ID del alumno a eliminar: "))
    confirmar = input(f"¿Estás seguro de eliminar al alumno con ID {id_alumno}? (s/n): ").strip().lower()

    if confirmar == "s":
        conexion = sqlite3.connect("alumnos.db")
        cursor = conexion.cursor()

        cursor.execute("DELETE FROM alumnos WHERE id = ?", (id_alumno,))

        if cursor.rowcount > 0:
            print("Alumno eliminado correctamente.")
        else:
            print("No se encontró ningún alumno con ese ID.")

        conexion.commit()
        conexion.close()
    else:
        print("Operación cancelada.")

except ValueError:
    print("El ID debe ser un número entero.")


ID del alumno a eliminar: 1
¿Estás seguro de eliminar al alumno con ID 1? (s/n): s
Alumno eliminado correctamente.


# 🔍 ¿Qué hace este código?

📌 Confirmamos antes de borrar (`s` para sí), evitando errores humanos.

📌 Verificamos si el ID existía con `cursor.rowcount`.

📌 Usamos `try-except` para manejar errores de entrada.

🎯 Podés intentar borrar un alumno inexistente o cancelar la operación.


# Desafío - Sistema seguro de gestión de alumnos

Te propongo un desafío integral: crear un programa que permita **gestionar una base de datos de alumnos** con todas las buenas prácticas que aprendiste.

---

🔧 ¿Qué tiene que hacer el programa?

1. Mostrar un menú con las siguientes opciones:
   - Agregar un nuevo alumno
   - Listar todos los alumnos
   - Modificar el email de un alumno
   - Eliminar un alumno (con confirmación)
   - Salir

2. Usar **consultas SQL seguras** con parámetros (`?`)

3. Validar entradas (por ejemplo, que el ID sea un número válido)

4. Mostrar mensajes claros de éxito o advertencia

5. Cuidar la integridad de los datos (usando `commit()` y validaciones)


🎯 Este ejercicio te prepara para el Proyecto Integrador del curso. ¡Vamos!


---

(Intenta resolverlo antes de mirar la posible solución que aparece más abajo!)

In [ ]:
# Escribe aqui tu programa

----
Esta es una posible solución al desafío propuesto.

# ✅ Solución - Sistema de gestión de alumnos con menú completo

In [ ]:
import sqlite3

# Función para conectar
def conectar():
    return sqlite3.connect("alumnos.db")

# Agregar un nuevo alumno
def agregar_alumno():
    nombre = input("Nombre: ").strip()
    try:
        edad = int(input("Edad: "))
        email = input("Correo electrónico: ").strip()

        with conectar() as con:
            con.execute("INSERT INTO alumnos (nombre, edad, email) VALUES (?, ?, ?)", (nombre, edad, email))
        print("Alumno agregado.")
    except ValueError:
        print("Edad inválida.")

# Listar todos los alumnos
def listar_alumnos():
    with conectar() as con:
        cursor = con.execute("SELECT * FROM alumnos")
        alumnos = cursor.fetchall()
        if alumnos:
            print("\nAlumnos registrados:")
            for id_, nombre, edad, email in alumnos:
                print(f"{id_}. {nombre} ({edad} años) – {email}")
        else:
            print("No hay alumnos registrados.")

# Modificar email
def modificar_email():
    try:
        id_alumno = int(input("ID del alumno: "))
        nuevo_email = input("Nuevo email: ").strip()
        with conectar() as con:
            cursor = con.execute("UPDATE alumnos SET email = ? WHERE id = ?", (nuevo_email, id_alumno))
            if cursor.rowcount > 0:
                print("Email actualizado.")
            else:
                print("No se encontró ese ID.")
    except ValueError:
        print("ID inválido.")

# Eliminar un alumno con confirmación
def eliminar_alumno():
    try:
        id_alumno = int(input("ID del alumno a eliminar: "))
        confirmar = input(f"¿Eliminar al alumno con ID {id_alumno}? (s/n): ").strip().lower()
        if confirmar == "s":
            with conectar() as con:
                cursor = con.execute("DELETE FROM alumnos WHERE id = ?", (id_alumno,))
                if cursor.rowcount > 0:
                    print("Alumno eliminado.")
                else:
                    print("No se encontró ese ID.")
        else:
            print("Operación cancelada.")
    except ValueError:
        print("ID inválido.")

# Menú principal
def menu():
    while True:
        print("\n--- Menú ---")
        print("1. Agregar alumno")
        print("2. Listar alumnos")
        print("3. Modificar email")
        print("4. Eliminar alumno")
        print("5. Salir")

        opcion = input("Elegí una opción: ")

        if opcion == "1":
            agregar_alumno()
        elif opcion == "2":
            listar_alumnos()
        elif opcion == "3":
            modificar_email()
        elif opcion == "4":
            eliminar_alumno()
        elif opcion == "5":
            print("¡Hasta la próxima!")
            break
        else:
            print("Opción inválida.")

# Ejecutamos el programa
menu()



--- Menú ---
1. Agregar alumno
2. Listar alumnos
3. Modificar email
4. Eliminar alumno
5. Salir
Elegí una opción: 1
Nombre: Ana
Edad: 45
Correo electrónico: ana@gmail.com
Alumno agregado.

--- Menú ---
1. Agregar alumno
2. Listar alumnos
3. Modificar email
4. Eliminar alumno
5. Salir
Elegí una opción: 2

Alumnos registrados:
2. Julián (19 años) – julian@mail.com
3. Dana (21 años) – dana@mail.com
4. Ana (45 años) – ana@gmail.com

--- Menú ---
1. Agregar alumno
2. Listar alumnos
3. Modificar email
4. Eliminar alumno
5. Salir
Elegí una opción: 5
¡Hasta la próxima!


# 🔍 ¿Qué hace este código?

📌 Cada operación está separada en una función clara y reutilizable.

📌 Todas las consultas usan parámetros (`?`) para evitar errores y proteger la base.

📌 Se validan las entradas del usuario con `try-except`.

📌 Se usan mensajes claros y confirmación antes de borrar datos importantes.

🎯 Este es un programa robusto y seguro, ideal para ser integrado o expandido en el proyecto final.


---

# 🎯 Lo que lograste en este cuaderno

Esta cuaderno te permitió reforzar tu conocimiento sobre bases de datos con un enfoque más **seguro, limpio y profesional**.

📌 Aplicaste operaciones CRUD completas: Crear, Leer, Actualizar y Eliminar  

📌 Usaste **consultas parametrizadas** para proteger tu base  

📌 Implementaste confirmaciones y validaciones de entrada  

📌 Separaste funciones para mejorar la organización del código  

📌 Practicaste un programa interactivo que guarda datos de forma persistente


🎯 Ya podés construir sistemas reales que administren información de manera confiable.

---


💡 Estamos llegando al Proyecto Final, cierre natural de tu recorrido y el momento para **crear algo propio, útil y funcional**.

¡Nos vemos en el próximo cuaderno!